# Variables

## Import libraries

In [ ]:
import os
import sys
from rdflib import URIRef

## Defining global variables

In [ ]:
# Existing files
ont_file_name = "ontology.ttl"
ruleset_file_name = "rules.pie"

# Created files during process
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"
facts_ttl_file_name = "facts_data.ttl"
implicit_to_facts_ttl_file_name = "implicit_to_facts.ttl"

# Existing folders
data_folder_name = "../../data/addresses"
mapping_folder_name = "mappings"

# Created folder during process
tmp_folder_name = "tmp_files"

# GraphDB repository name
facts_repository_name = "addresses_from_factoids"

# Definition of names of names graphes 
ontology_named_graph_name = "ontology"
facts_named_graph_name = "facts"
factoids_named_graph_name = "factoids"
permanent_named_graph_name = "permanent"
tmp_named_graph_name = "temporary"
inter_sources_name_graph_name = "inter_sources"

# URIs to access to GraphDB
graphdb_url = "http://localhost:7200"

py_code_folder_path = "./code"

## Processing global variables

* Obtaining absolute file paths from the relative paths given in the previous section
* Create a temporary folder if it doesn't already exist to store files to be deleted.

In [ ]:
tmp_folder = os.path.abspath(tmp_folder_name)
mapping_folder = os.path.abspath(mapping_folder_name)
data_folder = os.path.abspath(data_folder_name)

python_code_folder = os.path.abspath(py_code_folder_path)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
ont_file = os.path.abspath(ont_file_name)
ruleset_file = os.path.abspath(ruleset_file_name)
facts_ttl_file = os.path.join(tmp_folder, facts_ttl_file_name)
implicit_to_facts_ttl_file = os.path.join(tmp_folder, implicit_to_facts_ttl_file_name)

## Import des modules situés dans le dossier `code`

In [ ]:
# Calling up the `code` folder contains the python codes
sys.path.insert(1, python_code_folder)

import filemanagement as fm
import graphdb as gd
import attributeversioncomparisons as avc
import multisourcesprocessing as msp
import factoidscreation as fc
import timeprocessing as tp

## Creation of folders if they don't exist

In [ ]:
fm.create_folder_if_not_exists(tmp_folder)

### Creating the local directory in GraphDB
For the creation to work, GraphDB must be launched and therefore the URI given by `graphdb_url` must work. If the directory already exists, nothing is done.

In [ ]:
# Il se peut que la suppression d'un répertoire ne fonctionne pas donc pour éviter la suppresion au moment de la réinitialisation (suppression + (re)création)
# `allow_removal` doit valoir False et dans ce cas-là, le répertoire sera juste vidé.
allow_removal = False
disable_same_as = False

# gd.reinitialize_repository(graphdb_url, facts_repository_name, local_config_file, ruleset_name="owl2-rl-optimized", disable_same_as=disable_same_as, allow_removal=allow_removal)
gd.reinitialize_repository(graphdb_url, facts_repository_name, local_config_file, ruleset_file=ruleset_file, disable_same_as=disable_same_as, allow_removal=allow_removal)

## Local directory management

## Importing ontologies

In [ ]:
gd.load_ontologies(graphdb_url, facts_repository_name, [ont_file], ontology_named_graph_name)

## Definition of variables linked to sources

### Paris thoroughfares via Wikidata

* `wd` for "wikidata"
* `wdp_land` for "wikidata paris landmarks"
* `wdp_loc` for "wikidata paris locations"

In [ ]:
# Name of the directory where the factoid triples of Wikidata data are stored and constructed
wd_repository_name = "factoids_wikidata"

# CSV file to store the result of the selection query
wdp_land_csv_file_name = "wd_paris_landmarks.csv"
wdp_land_csv_file = os.path.join(data_folder, wdp_land_csv_file_name)

# CSV file to store the result of the selection query
wdp_loc_csv_file_name = "wd_paris_locations.csv"
wdp_loc_csv_file = os.path.join(data_folder, wdp_loc_csv_file_name)

# TTL file for structuring knowledge of the Paris thoroughfares
wdp_kg_file_name = "wd_paris.ttl"
wdp_kg_file = os.path.join(tmp_folder, wdp_kg_file_name)

# Final TTL files for Wikidata factoids
wdp_factoids_kg_file_name = "wd_paris_factoids.ttl"
wdp_factoids_kg_file = os.path.join(tmp_folder, wdp_factoids_kg_file_name)
wdp_permanent_kg_file_name = "wd_paris_permanent.ttl"
wdp_permanent_kg_file = os.path.join(tmp_folder, wdp_permanent_kg_file_name)

# Time interval of validity of the source (there is not end time)
wdp_time_description = {
    "start_time" : {"stamp":"2024-08-26T00:00:00Z","precision":"day","calendar":"gregorian"}
    }

### Nomenclature of Paris thoroughfares (Ville de Paris data)

The City of Paris data is made up of two sets:
* [names of current street rights-of-way](https://opendata.paris.fr/explore/dataset/denominations-emprises-voies-actuelles)
* [obsolete street names](https://opendata.paris.fr/explore/dataset/denominations-des-voies-caduques)

Current roads have a geometric right of way, unlike the old thoroughfares.

* `vpt` for ‘ville paris thoroughfares’
* `vpta` for ‘ville paris thoroughfares actuelles’.
* `vptc` for ‘ville paris thoroughfares caduques’.

In [ ]:
# Name of the directory where the factoid triples of Ville de Paris data are stored and constructed
vpt_repository_name = "factoids_ville_de_paris"

# CSV files containting data
vpta_csv_file_name = "denominations-emprises-voies-actuelles.csv"
vpta_csv_file = os.path.join(data_folder, vpta_csv_file_name)
vptc_csv_file_name = "denominations-des-voies-caduques.csv"
vptc_csv_file = os.path.join(data_folder, vptc_csv_file_name)

# TTL file for structuring knowledge of the Paris thoroughfares
vpt_kg_file_name = "voies_paris.ttl"
vpt_kg_file = os.path.join(tmp_folder, vpt_kg_file_name)

# Final TTL files for Ville de Paris factoids
vpt_factoids_kg_file_name = "vpt_factoids.ttl"
vpt_factoids_kg_file = os.path.join(tmp_folder, vpt_factoids_kg_file_name)
vpt_permanent_kg_file_name = "vpt_permanent.ttl"
vpt_permanent_kg_file = os.path.join(tmp_folder, vpt_permanent_kg_file_name)

# Time interval of validity of the source (there is not end time)
vpt_time_description = {
    "start_time" : {"stamp":"2024-02-10T00:00:00Z","precision":"day","calendar":"gregorian"}
    }

### Base Adresse Nationale (BAN)

Data from the [Base Adresse Nationale (BAN)](https://adresse.data.gouv.fr/base-adresse-nationale) (National Address Base), available [here](https://adresse.data.gouv.fr/data/ban/adresses/latest/csv)

bpa` for ‘BAN paris addresses’

In [ ]:
# Name of the directory where the factoid triples of BAN data are stored and constructed
bpa_repository_name = "factoids_ban"

# CSV file containting data
bpa_csv_file_name = "ban_adresses.csv"
bpa_csv_file = os.path.join(data_folder, bpa_csv_file_name)

# TTL file for structuring knowledge of Paris addresses
bpa_kg_file_name = "ban_adresses.ttl"
bpa_kg_file = os.path.join(tmp_folder, bpa_kg_file_name)

# Final TTL file for BAN factoids
bpa_factoids_kg_file_name = "ban_factoids.ttl"
bpa_factoids_kg_file = os.path.join(tmp_folder, bpa_factoids_kg_file_name)
bpa_permanent_kg_file_name = "ban_permanent.ttl"
bpa_permanent_kg_file = os.path.join(tmp_folder, bpa_permanent_kg_file_name)

# Time interval of validity of the source (there is not end time)
bpa_time_description = {
    "start_time" : {"stamp":"2024-01-01T00:00:00Z","precision":"day","calendar":"gregorian"}
    }

### OpenStreetMap (OSM)

Extracting data from OpenStreetMap

In [ ]:
# Name of the directory where the factoid triples of OSM data are stored and constructed
osm_repository_name = "factoids_osm"

# CSV files containting data
osm_csv_file_name = "osm_adresses.csv"
osm_csv_file = os.path.join(data_folder, osm_csv_file_name)
osm_hn_csv_file_name = "osm_hn_adresses.csv"
osm_hn_csv_file = os.path.join(data_folder, osm_hn_csv_file_name)

# TTL file for structuring knowledge of OSM addresses
osm_kg_file_name = "osm_adresses.ttl"
osm_kg_file = os.path.join(tmp_folder, osm_kg_file_name)

# Final TTL files for OSM factoids
osm_factoids_kg_file_name = "osm_factoids.ttl"
osm_factoids_kg_file = os.path.join(tmp_folder, osm_factoids_kg_file_name)
osm_permanent_kg_file_name = "osm_permanent.ttl"
osm_permanent_kg_file = os.path.join(tmp_folder, osm_permanent_kg_file_name)

# Time interval of validity of the source (there is not end time)
osm_time_description = {
    "start_time" : {"stamp":"2024-01-01T00:00:00Z","precision":"day","calendar":"gregorian"}
    }

### Integration of data from Geojson files

These files are derived from the vectorisation of maps of Paris:
* the revised Napoleonic cadatre of 1847 ;
* Andriveau’ plan of 1849 ;
* municipal plot plan of 1871 ;
* the Municipal Atlas map of 1888.

#### Global variables for importing data from Geojson files

In [ ]:
lang = "fr"
landmark_type = "Thoroughfare"
geojson_join_property = "name"
tmp_kg_file_name = "tmp_kg.ttl"
tmp_kg_file = os.path.join(tmp_folder, tmp_kg_file_name)

#### Napoleonic cadastre of 1847

In [ ]:
# Name of the directory where data factoid triples are stored and constructed
cn_1847_repository_name = "factoids_1847_cadastre_nap"

# Geojson file containting data
cn_1847_geojson_file_name = "1847_cadastre_nap.geojson"
cn_1847_geojson_file = os.path.join(data_folder, cn_1847_geojson_file_name)
cn_1847_kg_file_name = "cn_1847_kg.ttl"
cn_1847_kg_file = os.path.join(tmp_folder, cn_1847_kg_file_name)

# Final TTL files of factoids from the revised 1847 Napoleonic cadastre
cn_1847_factoids_kg_file_name = "cn_1847_factoids.ttl"
cn_1847_factoids_kg_file = os.path.join(tmp_folder, cn_1847_factoids_kg_file_name)
cn_1847_permanent_kg_file_name = "cn_1847_permanent.ttl"
cn_1847_permanent_kg_file = os.path.join(tmp_folder, cn_1847_permanent_kg_file_name)

cn_1847_geojson = fm.read_json_file(cn_1847_geojson_file)

# Description of the source within a dictionary
cn_1847_source_desc = {
    "lang" : "fr", 
    "label" : "Cadastre napoléonien de Gentilly de 1847",
    "publisher" : {
        "label": "Empire français"
        }
}

# Time interval of validity of the source
cn_1847_time_interval = {
    "start_time" : {"stamp":"1845-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
    "end_time" : {"stamp":"1850-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
}
cn_1847_geojson["source"] = cn_1847_source_desc
cn_1847_geojson["time"] = cn_1847_time_interval

#### Andriveau atlas

In [ ]:
# Name of the directory where data factoid triples are stored and constructed
an_1849_repository_name = "factoids_1849_andriveau"

# Geojson file containting data
an_1849_geojson_file_name = "1849_andriveau.geojson"
an_1849_geojson_file = os.path.join(data_folder, an_1849_geojson_file_name)
an_1849_kg_file_name = "an_1849_kg.ttl"
an_1849_kg_file = os.path.join(tmp_folder, an_1849_kg_file_name)

# Final TTL files of factoids from the 1849 Andriveau atlas
an_1849_factoids_kg_file_name = "an_1849_factoids.ttl"
an_1849_factoids_kg_file = os.path.join(tmp_folder, an_1849_factoids_kg_file_name)
an_1849_permanent_kg_file_name = "an_1849_permanent.ttl"
an_1849_permanent_kg_file = os.path.join(tmp_folder, an_1849_permanent_kg_file_name)

an_1849_geojson = fm.read_json_file(an_1849_geojson_file)

# Description of the source within a dictionary
an_1849_source_desc = {
    "lang" : "fr", 
    "label" : "Plan d'Andriveau de 1849",
    "publisher" : {
        "label": "Andriveau"
        }
}

# Time interval of validity of the source
an_1849_time_interval = {
    "start_time" : {"stamp":"1847-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
    "end_time" : {"stamp":"1851-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
}

an_1849_geojson["source"] = an_1849_source_desc
an_1849_geojson["time"] = an_1849_time_interval

#### 1871 municipal parcel map of Paris

In [ ]:
# Name of the directory where data factoid triples are stored and constructed
pm_1871_repository_name = "factoids_1871_plan_parcellaire_mun"

# Geojson file containting data
pm_1871_geojson_file_name = "1871_plan_parcellaire_mun.geojson"
pm_1871_geojson_file = os.path.join(data_folder, pm_1871_geojson_file_name)
pm_1871_kg_file_name = "pm_1871_kg.ttl"
pm_1871_kg_file = os.path.join(tmp_folder, pm_1871_kg_file_name)

# Final TTL file of factoids from the 1871 municipal parcel map
pm_1871_factoids_kg_file_name = "pm_1871_factoids.ttl"
pm_1871_factoids_kg_file = os.path.join(tmp_folder, pm_1871_factoids_kg_file_name)
pm_1871_permanent_kg_file_name = "pm_1871_permanent.ttl"
pm_1871_permanent_kg_file = os.path.join(tmp_folder, pm_1871_permanent_kg_file_name)

pm_1871_geojson = fm.read_json_file(pm_1871_geojson_file)

# Description of the source within a dictionary
pm_1871_source_desc = {
    "lang" : "fr",
    "label" : "Plan parcellaire municipal",
    "publisher" : {
        "label": "IIIe République"
        }
}

# Time interval of validity of the source
pm_1871_time_interval = {
    "start_time" : {"stamp":"1870-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
    "end_time" : {"stamp":"1872-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
}
pm_1871_geojson["source"] = pm_1871_source_desc
pm_1871_geojson["time"] = pm_1871_time_interval

#### 1888 Municipal Atlas of Paris

In [ ]:
# Name of the directory where data factoid triples are stored and constructed
am_1888_repository_name = "factoids_1888_atlas_municipal"

# Geojson file containting data
am_1888_geojson_file_name = "1888_atlas_municipal.geojson"
am_1888_geojson_file = os.path.join(data_folder, am_1888_geojson_file_name)
am_1888_kg_file_name = "am_1888_kg.ttl"
am_1888_kg_file = os.path.join(tmp_folder, am_1888_kg_file_name)

# Final TTL file of factoids from the 1888 Municipal Atlas plan
am_1888_factoids_kg_file_name = "am_1888_factoids.ttl"
am_1888_factoids_kg_file = os.path.join(tmp_folder, am_1888_factoids_kg_file_name)
am_1888_permanent_kg_file_name = "am_1888_permanent.ttl"
am_1888_permanent_kg_file = os.path.join(tmp_folder, am_1888_permanent_kg_file_name)

am_1888_geojson = fm.read_json_file(am_1888_geojson_file)

# Description of the source within a dictionary
am_1888_source_desc = {
    "lang" : "fr", 
    "label" : "Plan de l'atlas municipal de 1888",
    "publisher" : {
        "label": "Ville de Paris"
        }
}

# Time interval of validity of the source
am_1888_time_interval = {
    "start_time" : {"stamp":"1887-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
    "end_time" : {"stamp":"1889-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
}
am_1888_geojson["source"] = am_1888_source_desc
am_1888_geojson["time"] = am_1888_time_interval

## Final and iterative process

### Creating factoids in directories

For each source, factoids are created independently in separate directories

#### Ville de Paris


In [ ]:
fc.create_factoids_repository_ville_paris(graphdb_url, vpt_repository_name, tmp_folder,
                                          ont_file, ontology_named_graph_name,
                                          factoids_named_graph_name, permanent_named_graph_name,
                                          vpta_csv_file, vptc_csv_file, vpt_kg_file, vpt_time_description, lang=lang)

####  BAN


In [ ]:
fc.create_factoids_repository_ban(graphdb_url, bpa_repository_name, tmp_folder,
                                  ont_file, ontology_named_graph_name,
                                  factoids_named_graph_name, permanent_named_graph_name,
                                  bpa_csv_file, bpa_kg_file, bpa_time_description, lang=lang)

#### Wikidata


In [ ]:
# fc.get_data_from_wikidata(wdp_land_csv_file, wdp_loc_csv_file)
fc.create_factoids_repository_wikidata_paris(graphdb_url, wd_repository_name, tmp_folder,
                                             ont_file, ontology_named_graph_name,
                                             factoids_named_graph_name, permanent_named_graph_name,
                                             wdp_land_csv_file, wdp_loc_csv_file, wdp_kg_file, wdp_time_description=wdp_time_description, lang=lang)

#### OSM

In [ ]:
fc.create_factoids_repository_osm(graphdb_url, osm_repository_name, tmp_folder,
                                  ont_file, ontology_named_graph_name,
                                  factoids_named_graph_name, permanent_named_graph_name,
                                  osm_csv_file, osm_hn_csv_file, osm_kg_file, osm_time_description=osm_time_description, lang=lang)

#### Data from Geojson files

* Napoleonic cadastre of Gentilly (1847)
* Andriveau plan (1849)
* municipal parcel map of Paris (1871)
* municipal map of Paris (1888)

In [ ]:
fc.create_factoids_repository_geojson_states(graphdb_url, cn_1847_repository_name, tmp_folder, ont_file, ontology_named_graph_name,
                               factoids_named_graph_name, permanent_named_graph_name, cn_1847_geojson, geojson_join_property, cn_1847_kg_file, tmp_kg_file, landmark_type, lang)
fc.create_factoids_repository_geojson_states(graphdb_url, an_1849_repository_name, tmp_folder, ont_file, ontology_named_graph_name,
                               factoids_named_graph_name, permanent_named_graph_name, an_1849_geojson, geojson_join_property, an_1849_kg_file, tmp_kg_file, landmark_type, lang)
fc.create_factoids_repository_geojson_states(graphdb_url, pm_1871_repository_name, tmp_folder, ont_file, ontology_named_graph_name,
                               factoids_named_graph_name, permanent_named_graph_name, pm_1871_geojson, geojson_join_property, pm_1871_kg_file, tmp_kg_file, landmark_type, lang)
fc.create_factoids_repository_geojson_states(graphdb_url, am_1888_repository_name, tmp_folder, ont_file, ontology_named_graph_name,
                               factoids_named_graph_name, permanent_named_graph_name, am_1888_geojson, geojson_join_property, am_1888_kg_file, tmp_kg_file, landmark_type, lang)

### Insertion of factoids in the fact graph

In [ ]:
gd.remove_named_graph(graphdb_url, facts_repository_name, facts_named_graph_name)
gd.remove_named_graph(graphdb_url, facts_repository_name, inter_sources_name_graph_name)

#### Ville de Paris

In [ ]:
named_graph_name = "source_ville_de_paris"
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, vpt_repository_name,
                                           vpt_factoids_kg_file, vpt_permanent_kg_file,
                                           factoids_named_graph_name, permanent_named_graph_name, named_graph_name, facts_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, named_graph_name, facts_named_graph_name, inter_sources_name_graph_name)

#### Wikidata

In [ ]:
named_graph_name = "source_wikidata"
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, wd_repository_name,
                                           wdp_factoids_kg_file, wdp_permanent_kg_file,
                                           factoids_named_graph_name, permanent_named_graph_name, named_graph_name, facts_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, named_graph_name, facts_named_graph_name, inter_sources_name_graph_name)

#### BAN

In [ ]:
named_graph_name = "source_ban"
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, bpa_repository_name,
                                           bpa_factoids_kg_file, bpa_permanent_kg_file,
                                           factoids_named_graph_name, permanent_named_graph_name, named_graph_name, facts_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, named_graph_name, facts_named_graph_name, inter_sources_name_graph_name)

#### OSM

In [ ]:
named_graph_name = "source_osm"
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, osm_repository_name,
                                           osm_factoids_kg_file, osm_permanent_kg_file,
                                           factoids_named_graph_name, permanent_named_graph_name, named_graph_name, facts_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, named_graph_name, facts_named_graph_name, inter_sources_name_graph_name)

#### Data from Geojson files

* Napoleonic cadastre of Gentilly (1847)
* Andriveau plan (1849)
* municipal parcel map of Paris (1871)
* municipal map of Paris (1888)

In [ ]:
named_graph_name = "source_geojson"

msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, cn_1847_repository_name,
                                           cn_1847_factoids_kg_file, cn_1847_permanent_kg_file,
                                           factoids_named_graph_name, permanent_named_graph_name, named_graph_name, facts_named_graph_name)

msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, an_1849_repository_name,
                                           an_1849_factoids_kg_file, an_1849_permanent_kg_file,
                                           factoids_named_graph_name, permanent_named_graph_name, named_graph_name, facts_named_graph_name)

msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, pm_1871_repository_name,
                                           pm_1871_factoids_kg_file, pm_1871_permanent_kg_file,
                                           factoids_named_graph_name, permanent_named_graph_name, named_graph_name, facts_named_graph_name)

msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, am_1888_repository_name,
                                           am_1888_factoids_kg_file, am_1888_permanent_kg_file,
                                           factoids_named_graph_name, permanent_named_graph_name, named_graph_name, facts_named_graph_name)

msp.import_factoids_in_facts(graphdb_url, facts_repository_name, named_graph_name, facts_named_graph_name, inter_sources_name_graph_name)


### Version aggregation and change inference

In [ ]:
order_named_graph_name = "temporal_ordering"

facts_named_graph_uri = gd.get_named_graph_uri_from_name(graphdb_url, facts_repository_name, facts_named_graph_name)
inter_sources_name_graph_uri = gd.get_named_graph_uri_from_name(graphdb_url, facts_repository_name, inter_sources_name_graph_name)
tmp_named_graph_uri = gd.get_named_graph_uri_from_name(graphdb_url, facts_repository_name, tmp_named_graph_name)
order_named_graph_uri = gd.get_named_graph_uri_from_name(graphdb_url, facts_repository_name, order_named_graph_name)

#### Comparison of version values

In [ ]:
comparison_settings = {
    "geom_similarity_coef": 0.85,
    "geom_buffer_radius": 5,
    "geom_crs_uri": URIRef('http://www.opengis.net/def/crs/EPSG/0/2154'),
}
comp_named_graph_name = "version_comparisons"
comp_tmp_file_name = "version_comparisons.ttl"
comp_tmp_file = os.path.join(tmp_folder, comp_tmp_file_name)
avc.compare_attribute_versions(graphdb_url, facts_repository_name, comp_named_graph_name, comp_tmp_file, comparison_settings)

#### Inferring changes

In [ ]:
# Sort landmark versions and their attributes by time
msp.order_temporally_landmark_versions(graphdb_url, facts_repository_name, order_named_graph_uri, tmp_named_graph_uri)
msp.order_temporally_attribute_versions(graphdb_url, facts_repository_name, order_named_graph_uri, tmp_named_graph_uri)
msp.create_roots_and_traces_for_landmark_attribute_versions(graphdb_url, facts_repository_name, facts_named_graph_uri, inter_sources_name_graph_uri, tmp_named_graph_uri)

### Transfer information from factoids

In [ ]:
msp.transfer_version_values_to_roots(graphdb_url, facts_repository_name, facts_named_graph_uri)
msp.transfer_provenances_to_roots(graphdb_url, facts_repository_name, facts_named_graph_uri)
msp.transfer_crisp_time_instant_elements_to_roots(graphdb_url, facts_repository_name, facts_named_graph_uri)